In [15]:
# -*- coding: utf-8 -*
import numpy as np
import os
import tensorflow as tf
from flyai.processor.base import Base
from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
from tensorflow.python.ops import io_ops

#import create_dict
import model
from path import DATA_PATH  # 导入输入数据的地址

In [3]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import sys
from time import time
import json
import os
import platform
import random
import requests
import pandas as pd
import numpy
import numpy as np

from PIL import Image
from flyai.processor.base import Base
from flyai.processor.download import check_download
from flyai.utils.yaml_helper import Yaml
from flyai.utils import read_data

from path import DATA_PATH

import librosa
import scipy.io.wavfile as wavfile

In [4]:
def Csv(config,line=""):
    if line is "":
        line = True
    else:
        line = False
    train_path = check_download(config['train_url'], DATA_PATH, is_print=line)
    data = read_data.read(train_path)
    val_path = check_download(config['test_url'], DATA_PATH, is_print=line)
    val = read_data.read(val_path)
    return data,val

def load_csv(custom_source=None):
    yaml = Yaml()
    try:
        f = open(os.path.join(sys.path[0], 'train.json'))
        line = f.readline().strip()
    except IOError:
        line = ""

    postdata = {'id': yaml.get_data_id(),
                'env': line,
                'time': time(),
                'sign': random.random(),
                'goos': platform.platform()}

    try:
        servers = yaml.get_servers()
        r = requests.post(servers[0]['url'] + "/dataset", data=postdata)
        source = json.loads(r.text)
    except:
        source = None

    if source is None:
        trn,val = Csv({'train_url': os.path.join(DATA_PATH, "dev.csv"),'test_url': os.path.join(DATA_PATH, "dev.csv")}, line)
    elif 'yaml' in source:
        source = source['yaml']
        if custom_source is None:
            trn,val = Csv(source['config'], line)
        else:
            source = custom_source
    else:
        if not os.path.exists(os.path.join(DATA_PATH, "train.csv")) and not os.path.exists(
                os.path.join(DATA_PATH, "test.csv")):
            raise Exception("invalid data id!")
        else:
            trn,val = Csv({'train_url': os.path.join(DATA_PATH, "train.csv"),'test_url': os.path.join(DATA_PATH, "test.csv")}, line)
    print(source)
    return trn,val


def load_data(combine=True,summary=True):
    trn,val = load_csv()
    if combine:
        trn = pd.concat([trn,val])
    if summary:
        data_summary = trn.describe()
        for k in range(data_summary.shape[1]):
            print(list(data_summary.iloc[:,k]))
        for i in range(1,trn.shape[1]):
            print(trn.iloc[:,i].value_counts()[:10])
    return trn, val

In [5]:
trn,val = load_data()

{'id': 'SpeechCommand', 'type': 'csv', 'config': {'train_url': 'https://dataset.flyai.com/dataset/SpeechCommand/dev.zip', 'test_url': 'https://dataset.flyai.com/dataset/SpeechCommand/dev.zip'}}
[200, 100, 'audio/c5344cab965939d8f11ece248678eff6.wav', 2]
[200, 12, 'go', 24]
go         24
off        20
unknown    20
no         20
down       20
left       20
stop       20
on         14
right      12
up         12
Name: label, dtype: int64


In [6]:
trn

,wav,label
0,audio/8ce19853dcf57ebd430abffb6efe529d.wav,up
1,audio/dd855dfa20b1a85b1fff4a90b5cc7438.wav,no
2,audio/601d7b92c411511ccf21f5fe2443c787.wav,silence
3,audio/064490b507249f6d0d8fc8cf8d43076d.wav,off
4,audio/55e7f0e01b30de422f516b8ade928ddb.wav,unknown
5,audio/33d690fcc67cbe934a36e7f3e0020915.wav,off
6,audio/cc669b09c5800e09a31a48306a173841.wav,on
7,audio/939fc1c580534498b073c8514116b109.wav,silence
8,audio/b30f409423ac83f4c0f265715590d0fc.wav,down
9,audio/e56f1a4e2080f349f4ccbc129ae6dbb4.wav,unknown


In [7]:
class AudioProcessor(object):

    def prepare_processing_graph(self, model_settings):
        """Builds a TensorFlow graph to apply the input distortions.

        Creates a graph that loads a WAVE file, decodes it, scales the volume,
        shifts it in time, adds in background noise, calculates a spectrogram, and
        then builds an MFCC fingerprint from that.

        This must be called with an active TensorFlow session running, and it
        creates multiple placeholder inputs, and one output:

          - wav_filename_placeholder_: Filename of the WAV to load.
          - foreground_volume_placeholder_: How loud the main clip should be.
          - time_shift_padding_placeholder_: Where to pad the clip.
          - time_shift_offset_placeholder_: How much to move the clip in time.
          - background_data_placeholder_: PCM sample data for background noise.
          - background_volume_placeholder_: Loudness of mixed-in background.
          - mfcc_: Output 2D fingerprint of processed audio.

        Args:
          model_settings: Information about the current model being trained.
        """
        desired_samples = model_settings['desired_samples']
        self.wav_filename_placeholder_ = tf.placeholder(tf.string, [])
        wav_loader = io_ops.read_file(self.wav_filename_placeholder_)
        wav_decoder = contrib_audio.decode_wav(
            wav_loader, desired_channels=1, desired_samples=desired_samples)
        # Allow the audio sample's volume to be adjusted.
        self.foreground_volume_placeholder_ = tf.placeholder(tf.float32, [])
        scaled_foreground = tf.multiply(wav_decoder.audio,
                                        self.foreground_volume_placeholder_)
        # Shift the sample's start position, and pad any gaps with zeros.
        self.time_shift_padding_placeholder_ = tf.placeholder(tf.int32, [2, 2])
        self.time_shift_offset_placeholder_ = tf.placeholder(tf.int32, [2])
        padded_foreground = tf.pad(
            scaled_foreground,
            self.time_shift_padding_placeholder_,
            mode='CONSTANT')
        sliced_foreground = tf.slice(padded_foreground,
                                     self.time_shift_offset_placeholder_,
                                     [desired_samples, -1])
        # Mix in background noise.
        self.background_data_placeholder_ = tf.placeholder(tf.float32,
                                                           [desired_samples, 1])
        self.background_volume_placeholder_ = tf.placeholder(tf.float32, [])
        background_mul = tf.multiply(self.background_data_placeholder_,
                                     self.background_volume_placeholder_)
        background_add = tf.add(background_mul, sliced_foreground)
        background_clamp = tf.clip_by_value(background_add, -1.0, 1.0)
        # Run the spectrogram and MFCC ops to get a 2D 'fingerprint' of the audio.
        spectrogram = contrib_audio.audio_spectrogram(
            background_clamp,
            window_size=model_settings['window_size_samples'],
            stride=model_settings['window_stride_samples'],
            magnitude_squared=True)
        self.mfcc_ = contrib_audio.mfcc(
            spectrogram,
            wav_decoder.sample_rate,
            dct_coefficient_count=model_settings['dct_coefficient_count'])

    def get_data(self, filename, model_settings, time_shift, sess):
        """Gather samples from the data set, applying transformations as needed.

        When the mode is 'training', a random selection of samples will be returned,
        otherwise the first N clips in the partition will be used. This ensures that
        validation always uses the same samples, reducing noise in the metrics.

        Args:
          model_settings: Information about the current model being trained.
          background_frequency: How many clips will have background noise, 0.0 to
            1.0.
          background_volume_range: How loud the background noise will be.
          time_shift: How much to randomly shift the clips by in time.
          mode: Which partition to use, must be 'training', 'validation', or
            'testing'.
          sess: TensorFlow session that was active when processor was created.

        Returns:
          List of sample data for the transformed samples, and list of labels in
          one-hot form.
        """
        sample_count = 1
        data = np.zeros((sample_count, model_settings['fingerprint_size']))
        desired_samples = model_settings['desired_samples']
        # If we're time shifting, set up the offset for this sample.
        if time_shift > 0:
            time_shift_amount = np.random.randint(-time_shift, time_shift)
        else:
            time_shift_amount = 0
        if time_shift_amount > 0:
            time_shift_padding = [[time_shift_amount, 0], [0, 0]]
            time_shift_offset = [0, 0]
        else:
            time_shift_padding = [[0, -time_shift_amount], [0, 0]]
            time_shift_offset = [-time_shift_amount, 0]
        input_dict = {
            self.wav_filename_placeholder_: filename,
            self.time_shift_padding_placeholder_: time_shift_padding,
            self.time_shift_offset_placeholder_: time_shift_offset,
        }
        background_reshaped = np.zeros([desired_samples, 1])
        background_volume = 0
        input_dict[self.background_data_placeholder_] = background_reshaped
        input_dict[self.background_volume_placeholder_] = background_volume
        input_dict[self.foreground_volume_placeholder_] = 1
        # Run the graph to produce the output audio.
        tmp = sess.run(self.mfcc_, feed_dict=input_dict).flatten()
        data[0, :] = tmp
        return data

    def get_unprocessed_data(self, filename, model_settings):
        """Retrieve sample data for the given partition, with no transformations.

        Args:
          how_many: Desired number of samples to return. -1 means the entire
            contents of this partition.
          model_settings: Information about the current model being trained.
          mode: Which partition to use, must be 'training', 'validation', or
            'testing'.

        Returns:
          List of sample data for the samples, and list of labels in one-hot form.
        """
        desired_samples = model_settings['desired_samples']
        data = np.zeros((1, desired_samples))
        with tf.Session(graph=tf.Graph()) as sess:
            wav_filename_placeholder = tf.placeholder(tf.string, [])
            wav_loader = io_ops.read_file(wav_filename_placeholder)
            wav_decoder = contrib_audio.decode_wav(
                wav_loader, desired_channels=1, desired_samples=desired_samples)
            foreground_volume_placeholder = tf.placeholder(tf.float32, [])
            scaled_foreground = tf.multiply(wav_decoder.audio,
                                            foreground_volume_placeholder)
            input_dict = {wav_filename_placeholder: filename}
            input_dict[foreground_volume_placeholder] = 1
            data[0, :] = sess.run(scaled_foreground, feed_dict=input_dict).flatten()
        return data

In [8]:
audio_processor = AudioProcessor()

In [16]:
TENSORFLOW_MODEL_DIR = "best"

time_shift_ms = 100.0
sample_rate = 16000
clip_duration_ms = 1000
window_size_ms = 30.0
window_stride_ms = 10.0
dct_coefficient_count = 40
label_count = 12


def prepare_model_settings():
    """Calculates common settings needed for all models.

    Args:
      label_count: How many classes are to be recognized.
      sample_rate: Number of audio samples per second.
      clip_duration_ms: Length of each audio clip to be analyzed.
      window_size_ms: Duration of frequency analysis window.
      window_stride_ms: How far to move in time between frequency windows.
      dct_coefficient_count: Number of frequency bins to use for analysis.

    Returns:
      Dictionary containing common settings.
    """
    desired_samples = int(sample_rate * clip_duration_ms / 1000)
    window_size_samples = int(sample_rate * window_size_ms / 1000)
    window_stride_samples = int(sample_rate * window_stride_ms / 1000)
    length_minus_window = (desired_samples - window_size_samples)
    if length_minus_window < 0:
        spectrogram_length = 0
    else:
        spectrogram_length = 1 + int(length_minus_window / window_stride_samples)
    fingerprint_size = dct_coefficient_count * spectrogram_length
    return {
        'desired_samples': desired_samples,
        'window_size_samples': window_size_samples,
        'window_stride_samples': window_stride_samples,
        'spectrogram_length': spectrogram_length,
        'dct_coefficient_count': dct_coefficient_count,
        'fingerprint_size': fingerprint_size,
        'label_count': label_count,
        'sample_rate': sample_rate,
    }


In [17]:
model_settings = prepare_model_settings()

In [19]:
audio_processor.prepare_processing_graph(model_settings)

In [20]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.1
with tf.Session(config=config) as sess:
    data = audio_processor.get_data(os.path.join(DATA_PATH, 'audio/8ce19853dcf57ebd430abffb6efe529d.wav'), model_settings, 0, sess)

In [23]:
data

array([[-25.09768295,   4.66023445,   1.74488676, ...,   0.38340071,
         -0.05255827,   0.08426275]])

In [22]:
data.shape

(1, 3920)

In [ ]:
class Processor(Base):
    def __init__(self):
        super(Processor, self).__init__()
        self.audio_processor = AudioProcessor()
        self.model_settings = model.prepare_model_settings()
        self.audio_processor.prepare_processing_graph(self.model_settings)
        self.label_dict, self.label_dict_res = create_dict.load_label_dict()

    def input_x(self, wav):
        '''
        参数为csv中作为输入x的一条数据，该方法会被Dataset多次调用
        '''
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        config.gpu_options.per_process_gpu_memory_fraction = 0.1
        with tf.Session(config=config) as sess:
            data = self.audio_processor.get_data(os.path.join(DATA_PATH, wav), self.model_settings, 0, sess)
        return np.squeeze(data, axis=0)

    def input_y(self, label):
        '''
        参数为csv中作为输入y的一条数据，该方法会被Dataset多次调用
        '''
        label_count = self.model_settings['label_count']
        y = np.zeros((label_count,))
        if label in self.label_dict:
            y[self.label_dict[label]] = 1
        else:
            y[1] = 1
        return y

    def output_y(self, data):
        '''
        验证时使用，把模型输出的y转为对应的结果
        '''
        out_y = np.argmax(data)
        if out_y in self.label_dict_res:
            return self.label_dict_res[out_y]
        else:
            return self.label_dict_res[0]